In [ ]:
# TODO: REMOVE ME!
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

`Report v0.3`

In [ ]:
import os
os.environ["MPLCONFIGDIR"] = "/tmp"

In [ ]:
import cv2
import itertools
import numpy as np
import os
import pandas as pd
import random
from scipy.spatial.distance import cdist
from IPython.display import HTML, display
from plaster.run.calib.calib import RegPSF, approximate_psf
from plaster.run.job import JobResult
from plaster.run.plots import plots, plots_dev
from plaster.run.plots.plots_sigproc import plot_psfs, circle_locs, sigproc_v2_im, sigproc_v2_movie_from_df, sigproc_v2_im_from_df
from plaster.run.plots.plots_sigproc import wizard_xy_df, wizard_scat_df, wizard_raw_images
from plaster.run.run import RunResult
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.run.sigproc_v2.sigproc_v2_result import df_filter, radmat_from_df_filter, df_to_radmat, mean_non_dark_asr
from plaster.run.sigproc_v2.synth import Synth
from plaster.run.sigproc_v2 import bg
from plaster.tools.image import imops
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.ipynb_helpers.displays import hd, movie, md, h
from plaster.tools.log.log import error, debug
from plaster.tools.schema import check
from plaster.tools.utils import data
from plaster.tools.utils import utils
from plaster.tools.zap import zap
from plaster.tools.zplots import zplots
from plaster.tools.ipynb_helpers import displays
from plaster.run.nn_v2.nn_v2_worker import triangle_dyemat
z = zplots.setup()

In [ ]:
# REMOVE ME
# from plaster.run.base_result import enable_disk_memoize_from_notebook
# enable_disk_memoize_from_notebook()

In [ ]:
job = JobResult.from_context(dev_override="/erisyon/internal/jobs_folder/abbe7_1t")
run = job.sigproc_v2
hd("h1", f"Analyzing {job.job_folder}")

In [ ]:
hd("h1", f"Frame Quality")
displays.explanation("""
    Summary:
      * A per-frame (field, channel, cycle) measurement of quality.
    
    Definitions:
      * "Quality" is a metric of low-frequency spectral characteristics.
    
    Notes:
      * Uses a constant feature scale (not scaled by image dimension) and
        therefore might vary drastically between the ifferent image sizes
        that are recorded by different instruments.
""")
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    qdf = run.ims_import.qualities()
    qdf = qdf[(qdf.field_i < run.sigproc_v2.n_fields) & (qdf.channel_i == ch_i)].sort_values(["quality"])
    z.hist(qdf.quality, _size_x=800, _size_y=150, f_title=f"Quality distribution")

    row_iz = utils.ispace(0, len(qdf), 3)

    # COMBINE all images for common percentile calculations
    ims = np.concatenate([
        run.sigproc_v2.aln_ims[row.field_i, row.channel_i, row.cycle_i].flatten()
        for row in qdf.iloc[row_iz].itertuples()
    ])
    bot, top = np.percentile(ims, (50, 99.9))

    # SHOW example of worst, median, and best all using the same cspan
    hd("h3", f"Examples of frames by quality")
    with z(_cols=3, _cspan=(bot, top)):
        names = ("worst", "median", "best")
        for name, row in zip(names, qdf.iloc[row_iz].itertuples()):
            z.im(run.sigproc_v2.aln_ims[row.field_i, row.channel_i, row.cycle_i], f_title=f"{name} fl_i={row.field_i} cy_i={row.cycle_i}, qual={row.quality:.0f}")

In [ ]:
hd("h1", f"Field Quality")
displays.explanation("""
    Summary:
      * Per-field summaries of the "Quality" metric. (See above
        definitions and warnings.)
    
    Notes:
      * Each bar is the mean quality of all cycles for each field.
      * Channels are independent plots.
      
    Todo:
      * Add error bars
""")
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    qdf = run.ims_import.qualities()
    qdf = qdf[(qdf.field_i < run.sigproc_v2.n_fields) & (qdf.channel_i == ch_i)].sort_values(["quality"])
    mean_qual = qdf.groupby("field_i").mean().reset_index().sort_values("field_i")
    z.cols(
        mean_qual.quality,
        _size_x=800, _size_y=150,
        f_title=f"Mean quality distribution",
        f_x_axis_label="Field",
        f_y_axis_label="Mean quality",
        f_toolbar_location="above",
    )


In [ ]:
hd("h1", f"Field Alignment")
displays.explanation("""
    Summary:
      * Maximum shift in pixels required to align cycles for each field.
    
    Notes:
      * When a cycle fails to align it is typically dramatic and the shift will be
        very different from other fields.
      * The failures are typically caused by anomalies (contamination, etc) that
        confuses the aligner. Eventually the anomaly mask algorithm will be resurrected
        and will prevent many such failures to align.
""")
field_df = run.sigproc_v2.fields().copy()
field_df["alignment"] = np.sqrt(field_df.aln_x**2 + field_df.aln_y**2)
alignment = field_df.groupby("field_i").alignment.max().values
z.cols(alignment, f_x_axis_label="field_i", f_y_axis_label="Max shift in pixels", f_title="Max alignment distance by field")

In [ ]:
def features(run, ch_i=0, dyts=None):
    """
    Extract standard features for every peak
    """
    df = run.sigproc_v2.peaks()

    # Merge in stage metadata
    stage_df = (
        run.ims_import.metadata()[["field_i", "stage_x", "stage_y"]]
        .groupby("field_i")
        .mean()
    )
    df = pd.merge(left=df, right=stage_df, left_on="field_i", right_on="field_i")
    df["flowcell_x"] = df.stage_x + df.aln_x
    df["flowcell_y"] = df.stage_y + df.aln_y

    sig = run.sigproc_v2.sig()[:, ch_i, :]
    assert sig.shape[0] == len(df)

    asr = run.sigproc_v2.aspect_ratio()[:, ch_i, :]
    assert asr.shape[0] == len(df)

    has_neighbor_stats = run.sigproc_v2.has_neighbor_stats()
    if has_neighbor_stats:
        nei = run.sigproc_v2.neighborhood_stats()
        assert nei.shape[0] == len(df)

    # Convenience aliases
    n_cycles = run.sigproc_v2.n_cycles
    n_peaks = df.peak_i.max() + 1
    im_mea = run.ims_import.dim

    # "Lifespan" is the cycles over which a peak is "on". Abbrevaited "lif"
    # Use the cosine distance to determine lif_len
    dyts1, _ = triangle_dyemat(n_cycles, n_dyes=1, include_nul_row=False)
    dyt1_dists = cdist(sig, dyts1, 'cosine')
    lif_len = np.argmin(dyt1_dists, axis=1) + 1  # Add one because we removed the nul row

    row_iz, col_iz = np.indices(sig.shape)
    sig_lif = np.where(col_iz < lif_len[:, None], sig, np.nan)
    asr_lif = np.where(col_iz < lif_len[:, None], asr, np.nan)

    # "afl" stands for "afterlife"
    sig_afl = np.where(col_iz >= lif_len[:, None], sig, np.nan)

    if has_neighbor_stats:
        nei_men_lif = np.where(col_iz < lif_len[:, None], nei[:, :, 0], np.nan)
        nei_std_lif = np.where(col_iz < lif_len[:, None], nei[:, :, 1], np.nan)
        nei_med_lif = np.where(col_iz < lif_len[:, None], nei[:, :, 2], np.nan)
        nei_iqr_lif = np.where(col_iz < lif_len[:, None], nei[:, :, 3], np.nan)
    else:
        nei_men_lif = np.zeros((n_peaks, n_cycles))
        nei_std_lif = np.zeros((n_peaks, n_cycles))
        nei_med_lif = np.zeros((n_peaks, n_cycles))
        nei_iqr_lif = np.zeros((n_peaks, n_cycles))

    df["radius"] = np.sqrt(
        (df.aln_x - im_mea // 2) ** 2 + (df.aln_y - im_mea // 2) ** 2
    )

    with utils.np_no_warn():
        df["lif_len"] = lif_len
        df["lif_med"] = np.nanmedian(sig_lif, axis=1)
        df["lif_men"] = np.nanmean(sig_lif, axis=1)
        df["lif_std"] = np.nanstd(sig_lif, axis=1)
        df["lif_iqr"] = np.subtract(*np.nanpercentile(sig_lif, [75, 25], axis=1))
        df["lif_max"] = np.nanmax(sig_lif, axis=1)
        df["lif_min"] = np.nanmin(sig_lif, axis=1)
        df["lif_rng"] = df.lif_max - df.lif_min

        df["afl_med"] = np.nanmedian(sig_afl, axis=1)
        df["afl_men"] = np.nanmean(sig_afl, axis=1)
        df["afl_std"] = np.nanstd(sig_afl, axis=1)
        df["afl_iqr"] = np.subtract(*np.nanpercentile(sig_afl, [75, 25], axis=1))
        df["afl_max"] = np.nanmax(sig_afl, axis=1)
        df["afl_min"] = np.nanmin(sig_afl, axis=1)
        df["afl_rng"] = df.afl_max - df.afl_min

        df["nei_men"] = np.nanmean(nei_men_lif, axis=1)
        df["nei_std"] = np.nanmean(nei_std_lif, axis=1)
        df["nei_med"] = np.nanmean(nei_med_lif, axis=1)
        df["nei_iqr"] = np.nanmean(nei_iqr_lif, axis=1)

        df["sig_med"] = np.median(sig, axis=1)
        df["sig_men"] = np.mean(sig, axis=1)
        df["sig_std"] = np.std(sig, axis=1)
        df["sig_iqr"] = np.subtract(*np.nanpercentile(sig, [75, 25], axis=1))
        df["sig_max"] = np.max(sig, axis=1)
        df["sig_min"] = np.min(sig, axis=1)
        df["sig_rng"] = df.sig_max - df.sig_min

        df["asr_med"] = np.nanmedian(asr_lif, axis=1)
        df["asr_std"] = np.nanstd(asr_lif, axis=1)
        df["asr_max"] = np.nanmax(asr_lif, axis=1)
        df["asr_cy0"] = asr[:, 0]

    return df


In [ ]:
unfilt_fea_df = features(run)
n_peaks = unfilt_fea_df.peak_i.max() + 1
n_cycles = run.sigproc_v2.n_cycles
sig = run.sigproc_v2.sig()

hd("h1", f"Aspect ratios @ cycle=0, all channels")
displays.explanation("""
    Summary:
      * Left-side:
          Distribution of peak aspect-ratios on first cycle (a proxy for collisions).
          The red-line is a threshold used in later filters.
      * Right-side:
          Distribution of maximum ASR for each peak over its lifespan (on cycles).
    
    Definition:
      * "Aspect Ratio" (abbrevatied as "ASR") is the ratio of the longest to the
        shortest axis of a peak's pixels.
    
    Notes:
      * Currently filters only use ASR at cycle 0. A proposal has been made
        to convert this to the mean of all cycles so that we might detect
        situations where a collision comes in and out of a cycle.
      * The ASR Threshold was determined by simulating two peaks moving
        towards each other and findind the point at which the aspect ratio
        is a good predictor of a collision. This point was determined
        informally but probably wouldn't change much under a stricter definition.
""")
asr_thresh = run.sigproc_v2.asr_threshold()

asr = unfilt_fea_df.asr_cy0
asr_mask = asr < asr_thresh
n_keep = asr_mask.sum()
fea_df = unfilt_fea_df[asr_mask]
# fea_df from here forward is aspect_ratio filtered

hd("h2", f"Keep {100 * n_keep / n_peaks:.1f}%, Reject {100 * (n_peaks-n_keep) / n_peaks:.1f}%")
_bins = (1, 3, 200)
with z(_cols=2, _range=z.hist_range(asr, _bins=_bins), _bins=_bins, f_x_axis_label="ASR", f_y_axis_label="# peaks"):
    z.hist(asr, _vertical=asr_thresh, f_title="Distr. Aspect-ratio at cycle=0")
    z.hist(unfilt_fea_df.asr_max, f_title="Dist. Aspect-ratio max over lifespan cycles")

In [ ]:
hd("h1", f"Cycle Balance")
displays.explanation("""
    Summary:
      * Shows the relative correction that is applied to each cycle.
    
    Definitions:
      * "Relative Brightness": Each cycle's median peak intensity is computed
        using any peak that is "alive" during that cycle.
        1.0 = mean cycle brightness.
    
    Notes:
      * The mean of cycle brightnesses is found and used to normalized the y axis.
        So 1.0 is the "mean cycle intensity". "1.4" would mean that a given cycle
        was 40% brighter than the mean cycle.
""")

sig_ch0 = sig[fea_df.peak_i, 0, :]
sig_ch0_lif = np.where(np.arange(n_cycles)[None, :] < fea_df.lif_len[:, None], sig_ch0, np.nan)
cy_bal = np.nanmean(sig_ch0_lif, axis=0)
cy_bal /= np.median(cy_bal)
bal_sig = sig / cy_bal

cum_n_dark = np.zeros((n_cycles,))
for cy_i in range(n_cycles):
    cum_n_dark[cy_i] = (fea_df.lif_len <= cy_i).sum()

# TODO: Multi-channel
with z(_cols=2):
    z.cols(
        cy_bal,
        f_title="Cycle balance",
        f_x_axis_label="Cycle",
        f_y_axis_label="Relative brightness (1.0 = mean)",
    )


In [ ]:
hd("h1", f"Filters and Features")
displays.explanation("""
    Summary:
      * Shows distrbution of peaks by various features some of which are used
        for filtering.
    
    Definitions:
      * "Aspect Ratio": (abbrevatied as "ASR") is the ratio of the longest to the
        shortest axis of a peak's pixels.
      * "Dropped after cycle X": Determined by the lifespan metric
        (ie, a binary step fitter)
      * "Remainders": Peaks that were still bright on the last cycle
    
    Notes:
      * Currently ASR is the only filter and is applied to most subsequent plots.
""")

n_asr_pass, n_asr_reject = asr_mask.sum(), n_peaks - asr_mask.sum()

bars = [
    [n_asr_pass, n_asr_reject],
    [(fea_df.lif_len == cy_i).sum() for cy_i in range(1, n_cycles+1)],
]
labels = [
    [f"asr_pass", f"asr_reject"],
    [(f"dropped after cycle {cy_i}" if cy_i < (n_cycles-1) else "remainders") for cy_i in range(0, n_cycles)],
]
z.count_stack(bars, labels, _size=(1000, 100))


In [ ]:
hd("h1", f"Lifespans")
displays.explanation("""
    Summary:
      * Shows distrbution of peak lifespans
    
    Definitions:
      * "Lifespan": The number of cycles a peak is "on".
      * "Afterlife" (abbrev. "afl"): The values of cycles after the peak is
        declated off.
    
    Notes:
      * Lifespan is determined by a binary step function fitter using 
        cosine distance. It is not particularly sensitive to the
        brightness of the spot (ie, count, etc.)
""")

with z(_cols=3):
    with z(_range=z.hist_range(fea_df.lif_med)):
        with z(f_x_axis_label="median intensity (rad. units) of lifespan", f_y_axis_label="# peaks"):
            z.hist(
                fea_df.lif_med,
                f_title="Distr. of lifespan medians (ch 0)",
            )

            z.hist(
                fea_df[fea_df.lif_len < n_cycles].lif_med,
                f_title="Distr. of lifespan meds. (ch 0) excluding remainders",
            )

            z.hist(
                fea_df.afl_med,
                f_title="Distr. of after-life meds.",
            )

    z.hist(
        fea_df.lif_len,
        _bins=(0, run.sigproc_v2.n_cycles, run.sigproc_v2.n_cycles),
        f_title="Distr. of lifespan lengths (ch 0)",
        f_x_axis_label="lifespan in cycles",
        f_y_axis_label="# peaks",
    )

    cum_n_dark = np.zeros((n_cycles,))
    for cy_i in range(n_cycles):
        cum_n_dark[cy_i] = (fea_df.lif_len <= cy_i).sum()
    z.cols(
        100 * cum_n_dark / n_peaks,
        f_title="Cummulate # peaks dropped to dark (ch 0)",
        f_x_axis_label="cycle",
        f_y_axis_label="% peaks dark by cycle",
        _range_y=(0, 100)
    )

In [ ]:
hd("h1", f"Monotonicity")
displays.explanation("""
    Summary:
      * Left-side:
          - Distribution of "monotonic metric".
          - Red line is the "monotonic threshold"
      * Right-side:
          - Sampling of peaks where "monotonic metric" exceeds
            the "monotonic threshold"
            
    Definitions:
      * "Monotonic metric" is the maximum increase in radiometry intensity over
        the lifespan divided by the mean radiometric intensity of the lifespan.
      * "Monotonic threshold" is chosen arbitrarily to be 1.0.
    
    Filters applied:
      * aspect ratio
      * cycle balance
    
    Notes:
      * The metric is noisy for lifespans under 5 cycles and therefore
        is only shown for peaks with lifespans >= 5 cycles.
      * The units are set by the mean of each row so 1.0 means that there
        was some rise equal a full value of the row. For counts > 1, this
        will under-estimate the monotonicity as higher counts will lead to
        higher intensityies and thus an unfarly larger increase for a row
        to be declared "non-monotonic".
""")

sig0 = bal_sig[fea_df.peak_i, 0, :]
bot, top = np.percentile(sig0, (50, 99))
_, col_iz = np.indices(sig0.shape)
sig0_lif = np.where(col_iz < fea_df.lif_len[:, None], sig0, np.nan)
monotonic_threshold = 1.0
with utils.np_no_warn():
    d = np.diff(sig0_lif, axis=1)
    maxs_diff = np.nanmax(d, axis=1)
    mens = np.nanmean(sig0_lif, axis=1)
    monotonic_metric = maxs_diff / mens
    lif_gt_4_mask = fea_df.lif_len > 4
    n_peaks_lif_gt_4 = lif_gt_4_mask.sum()
    monotonic_metric_lif_gt_4 = maxs_diff[lif_gt_4_mask] / mens[lif_gt_4_mask]
    n_non_monotonic_lif_gt_4 = (monotonic_metric_lif_gt_4 > monotonic_threshold).sum()
    hd("h2", f"% Non-monotonic (of peaks with > 4 lifespan): {100 * n_non_monotonic_lif_gt_4 / n_peaks_lif_gt_4:.1f}%")
    mask = (maxs_diff / mens > monotonic_threshold) & lif_gt_4_mask

    with z(_cols=2):
        z.hist(
            monotonic_metric_lif_gt_4,
            _bins=(0, 3, 200),
            f_title="Distr. of monotonic metric (peaks w/ life > 4)",
            f_x_axis_label="max. rad. inten. gain in lifespan / lif_mean",
            f_y_axis_label="# peaks",
            _vertical=monotonic_threshold
        )
        
        mdf = fea_df.sort_values(["peak_i"])[mask]
        _hover_rows=dict(peak_i=mdf.peak_i, field_i=mdf.field_i)
        z.im_clus(
            sig0[mask],
            _n_samples=500,
            _cspan=(bot, top),
            f_title="Sample of sigfinicantly non-monotonic rows",
            f_x_axis_label="# peaks",
            _hover_rows=_hover_rows,
        )

In [ ]:
hd("h1", f"Heatmaps")
displays.explanation("""
    Summary:
      * Heatmaps of random subsample of peaks. Rows are peaks, Columns are Cycles
            
    Definitions:
      * "Clustered by row": heirarchical clustering that puts similar
        row patterns together
      * "Sorted by mean life. intensity": Sorted using the mean of the
        intensity during a peak's lifespan.
      * "Signal sorted by (lif_len, lif_mean)": Sort first by the
        assigned lifespan and then within that sort by the mean intensity
        during the lifespan.
      * "Signal sorted by (field_i, lif_len, lif_mean)": Same but with field
        as the first sort axis.
    
    Filters applied:
      * aspect ratio
      * cycle balance
      
    Notes:
      * This images may change on each execution as they involve random sampling.
""")

for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    bot, top = np.percentile(bal_sig, (50, 99))
    with z(_cols=2, _size=500, _cspan=(bot, top)):
        mdf = fea_df.sort_values(["peak_i"])
        _hover_rows=dict(peak_i=mdf.peak_i, field_i=mdf.field_i)
        axis_labels = dict(f_x_axis_label="cycle", f_y_axis_label="peaks")
        z.im_clus(
            bal_sig[mdf.peak_i, ch_i, :],
            _hover_rows=_hover_rows,
            _n_samples=500,
            f_title="Signal clustered by row",
            **axis_labels
        )

        mdf = fea_df.sort_values(["sig_men"])
        z.im(
            bal_sig[mdf.peak_i, ch_i, :],
            _hover_rows=_hover_rows,
            f_title="Signal sorted by mean life. intensity",
            **axis_labels
        )

        mdf = fea_df.sample(1000).sort_values(["lif_len", "lif_men"])
        z.im(
            bal_sig[mdf.peak_i, ch_i, :],
            _hover_rows=dict(peak_i=mdf.peak_i, field_i=mdf.field_i),
            f_title="Signal sorted by (lif_len, lif_mean)",
            **axis_labels
        )

        mdf = fea_df.sample(1000).sort_values(["field_i", "lif_len", "lif_men"])
        z.im(
            bal_sig[mdf.peak_i, ch_i, :],
            _hover_rows=dict(peak_i=mdf.peak_i, field_i=mdf.field_i),
            f_title="Signal sorted by (field_i, lif_len, lif_mean)",
            **axis_labels
        )

In [ ]:
hd("h1", f"Signal to Noise Ratios")
displays.explanation("""
    Summary:
      * Distribution of "Signal to Noise Ratio".
            
    Definitions:
      * "Signal to Noise Ratio" is the radiometry signal divided
        by the "Noise Estimate"
      * "Noise Estimate" is standard deviation of the residuals
        of the fit signal peak to the actual peak.
    
    Filters applied:
      * aspect ratio
    
    Notes:
      * There will always be a strong peak with mean at zero. These
      are the dark cycles.
""")

for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    _snr = run.sigproc_v2.snr()[fea_df.peak_i, ch_i, :]
    z.hist(_snr, f_title=f"Distr. of SNR channel={ch_i}", f_x_axis_label="SNR", f_y_axis_label="n_peaks")

In [ ]:
hd("h1", f"Peak Feature Relationships")
displays.explanation("""
    Summary:
      * All-feature to all-feature scatters. Each blue dot is a randomly
        sampled peak.
            
    Definitions:
      * field_i: Field index
      * aln_x, _y: Aligned coordinate in pixels relative to field
      * flowcell_x, _y: Field stage coordinate (in microns?)
        plus aln_x, aln_y (in pixels). Note that this is adding apples
        and oranges (need a portable conversion factor)!
      * radius: Distance in pixels from center of field.
      * lif_len: Lifespan of peak in cycles
      * lif_med: Median rad. intensity during lifespan
      * lif_iqr: IQR of rad. intensity during lifespan
      * afl_med: Median rad. intensity after the the lifespan (afterlife)
      * afl_iqr: IQR of rad. intensity after the the lifespan (afterlife)
      * nei_med: Median of neighborhood pixels (sometimes disabled)
      * nei_iqr: IQR of neighborhood pixels (sometimes disabled)
      * sig_med: Median of rad. intensity over all cycles (compare the lif_med)
      * sig_iqr: IQR of rad. intensity over all cycles
      * asr_cy0: Aspect ratio (ASR) at cycle 0
      * asr_max: Max. aspect ratio through lifespan
    
    Filters applied:
      * aspect ratio
      * lifespan < n_cycles (ie, no remainders)
    
    Notes:
      * 1000 peaks are drawn randomly. All plots use the same 1000 peaks.
      * A conversion factor from pixels to microns is needed for 
        a correct value of flowcell_x, _y.
      * The neighborhood measurement is not always availble.
""")

mask = fea_df.lif_len < n_cycles
mdf = fea_df[mask]
cols = [
    "field_i", "aln_y", "aln_x", "flowcell_y", "flowcell_x", "radius",
    "lif_len", "lif_med", "lif_iqr",
    "afl_med", "afl_iqr",
    "nei_med", "nei_iqr",
    "sig_med", "sig_iqr",
    "asr_cy0", "asr_max"
]
mdf = mdf.sample(1000)
with z(_cols=len(cols), _notools=True, _noaxes=True, _size=70, alpha=0.1):
    for yi, col_y in enumerate(cols):
        for xi, col_x in enumerate(cols):
            f_title = col_x if yi == 0 else ""
            f_y_axis_label = col_y if xi == 0 else ""
            z.scat(x=mdf[col_x], y=mdf[col_y], f_title=f_title, f_y_axis_label=f_y_axis_label)

In [ ]:
hd("h1", f"Calibration")
displays.explanation("""
    Summary:
      * PSF and Regional Illumination Stats from calibration
            
    Definitions:
      * "PSF": The regional Point-Spread-Function.
      * "Regional Illumination Balance": The foreground illumination balance found regionally
      * "Calibration": When an independent 1-count experiment is used to measeure PSF and Illum. balance.
      * "Self-Calibration": When the run itself is used to estimate these parameters.
    
    Filters applied:
      * None
""")

print(f"Uses self-calibration: {run.sigproc_v2.params.self_calib}")
print(f"Uses calibration_file: {run.sigproc_v2.params.calibration_file}")
print(f"Uses instrument_identity: {run.sigproc_v2.params.instrument_identity}")

for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    with z(_cols=3, _size=250):
        reg_psf = run.sigproc_v2.calib.reg_psf(ch_i=ch_i)
        check.t(reg_psf, RegPSF)
        psf_ims = reg_psf.render()
        plot_psfs(psf_ims, scale=3.0, f_title=f"Regional PSF", _noaxes=True, _notools=True, _zplots_context=z)

        z.cols(
            np.max(reg_psf.params[ch_i][:, :, 0:2], axis=2).flatten(),
            f_x_axis_label="Region #",
            f_y_axis_label="peak size",
            f_title="Regional PSF peak size",
        )

        illum = run.sigproc_v2.calib.reg_illum().interp(ch_i)
        z.im(1.0 / illum, f_title="Regional Illumination Balance Map", _cspan=(0, 1.0))


In [ ]:
hd("h1", f"Signal distributions")
displays.explanation("""
    Summary:
      * Signal distributions.
            
    Filters applied:
      * Aspect Ratio
      
    Notes:
      * The distr. plots show:
          - gray line for the (0-99) percentiles
          - black line for the (20-75) percentiles ("IQR")
          - white tick for the median
      * The red vertical line is the same in all plots and is simply
        a visual reference set to a guess of the beta parameter.
""")

n_peaks_per_field = run.sigproc_v2.peaks().groupby("field_i").count().field_peak_i
n_peaks_max = np.max(n_peaks_per_field)
n_fields = run.sigproc_v2.n_fields
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    sig_ch = bal_sig[fea_df.peak_i, ch_i, :]
    sig_ch_lif = np.where(np.arange(n_cycles)[None, :] < fea_df.lif_len[:, None], sig_ch, np.nan)

    dark = run.sigproc_v2.dark_estimate(ch_i=ch_i)
    sig_ch = bal_sig[fea_df.peak_i, ch_i, :]
    beta_est = np.median(sig_ch[sig_ch > dark])

    bot, top = np.nanpercentile(sig_ch, (0.1, 99.0))
    with z(_cols=3, _bins=(bot, top, 200), _range_x=(bot, top)):
        z.hist(
            sig_ch,
            _vertical=beta_est,
            f_x_axis_label="Signal (Rad. units)",
            f_title="Signal distributions, all cycles, all fields"
        )
        
        with z(_range=z.hist_range(sig_ch[:, -1])):
            z.hist(
                sig_ch[:, 0],
                _vertical=beta_est,
                f_x_axis_label="Signal (Rad. units)",
                f_title="Signal distributions, cycle=0, all fields"
            )

            z.hist(
                sig_ch[:, -1],
                _vertical=beta_est,
                f_x_axis_label="Signal (Rad. units)",
                f_title="Signal distributions, cycle=last, all fields"
            )

    sig_ch_cy0_by_field = np.full((n_fields, n_peaks_max), np.nan)
    for fl_i in range(n_fields):
        fl_mask = fea_df.field_i == fl_i
        _sig_ch_cy0 = sig_ch[fl_mask, 0]
        sig_ch_cy0_by_field[fl_i, 0:len(_sig_ch_cy0)] = _sig_ch_cy0

    with z(_cols=2):
        z.distr(
            sig_ch_lif.transpose(1,0),
            _vertical=beta_est,
            _percentiles=(0, 25, 50, 75, 99),
            _nogrid=True,
            f_x_axis_label="Signal",
            f_y_axis_label="Cycle",
            f_title="Distr. of life signal by cycle, all fields"
        )

        z.distr(
            sig_ch_cy0_by_field,
            _vertical=beta_est,
            _percentiles=(0, 25, 50, 75, 99),
            _nogrid=True,
            f_x_axis_label="Signal",
            f_y_axis_label="Field",
            f_title="Distr. of signal @ cycle=0, by field"
        )


In [ ]:
hd("h1", f"Peak density")
displays.explanation("""
    Summary:
      * Peak density per field.
      
    Notes:
      * Pay attention to the axes of the X/Y coords as it can occur
        that a limited number of fields are all on the same row/col
        and therefore the axis dimensions will be pure noise.
""")

n_peaks_per_field = run.sigproc_v2.peaks().groupby("field_i").count().field_peak_i
hd("h2", f"<i>Mean pixels per peak:</i> {run.ims_import.dim**2 / np.mean(n_peaks_per_field):.1f}")

field_df = run.ims_import.metadata()[["field_i", "stage_x", "stage_y"]].groupby("field_i").mean().join(n_peaks_per_field)
field_df["size"] = field_df.field_peak_i * 0.03

with z(_cols=2):
    z.cols(
        n_peaks_per_field,
        f_title="# peaks per field",
        f_x_axis_label="field",
        f_y_axis_label="n_peaks",
    )
    z.scat(
        source=field_df, x="stage_x", y="stage_y", size="size",
        f_title="# peaks per field in stage X/Y coords.",
        f_x_axis_label="stage x (microns?)",
        f_y_axis_label="stage y (microns?)",
    )


In [ ]:
hd("h1", f"Movies")
displays.explanation("""
    Summary:
      * Aligned movies.
        - Left: unfiltered
        - Right: with bandpass filter
      
    Filters applied:
      * Aspect ratio (The circles that are drawn)
""")


qdf = run.ims_import.qualities()

for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    qdf = qdf[(qdf.field_i < run.sigproc_v2.n_fields) & (qdf.channel_i == ch_i)]
    mean_qdf = qdf.groupby("field_i").mean().reset_index().sort_values("quality")
    worst_fl_i = int(mean_qdf.iloc[0].field_i.astype(int))
    median_fl_i = int(mean_qdf.iloc[len(mean_qdf) // 2].field_i.astype(int))

    # std_qdf = qdf.groupby("field_i").std().reset_index().sort_values("quality")
    # most_variable_fl_i = int(std_qdf.iloc[-1].field_i.astype(int))
    
    def movies(fl_i, description):
        hd("h3", f"Field={fl_i} ({description})")
        top_unfilt = np.percentile(run.sigproc_v2.aln_unfilt_ims[0,ch_i,0], 99.9)
        top_filt = np.percentile(run.sigproc_v2.aln_ims[0,ch_i,0], 99.9)
        sigproc_v2_movie_from_df(run, fea_df, fl_i=fl_i, _cspan=(0, top_unfilt), draw_unfilt=True, draw_filt=True, scale_filt=top_unfilt/top_filt)
        
    movies(worst_fl_i, "Worst quality")
    # movies(most_variable_fl_i, "Most variable quality")
    movies(median_fl_i, "Median quality")
    

In [ ]:
n_cycles = run.ims_import.n_cycles

hd("h1", f"Pixel Intensity Distributions (by cycle @ 5 random fields)")
displays.explanation("""
    Summary:
      * Distributions of pixels on 5 randomly chosen fields
      
    Filters applied:
      * None
    
    Notes:
      * The foreground pixels is somewhat arbitraily defined and
        may pick up a considerable number of background pixels.
      * The red line is set arbitrarily at 500 as a job-to-job reference.
""")

for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    n_samples = 10000
    bg_samps = np.zeros((n_cycles, n_samples))
    fg_samps = np.zeros((n_cycles, n_samples))
    idx = np.arange(run.ims_import.dim**2)
    cy_labels = [f"cy {cy_i}" for cy_i in range(n_cycles)]
    for fl_i in utils.ispace(0, run.ims_import.n_fields, 5):
        for cy_i in range(n_cycles):
            im = run.ims_import.ims[fl_i, ch_i, cy_i]
            bg_im, fg_mask = bg.background_extract(im, approximate_psf(), dilate=0)
            _idx = data.subsample(idx[fg_mask.flatten()], n_samples)
            bg_samps[cy_i, :] = data.subsample(bg_im.flatten(), n_samples)
            fg_samps[cy_i, :] = im.flatten()[_idx]

    with z(_cols=3):
        top = np.nanpercentile(fg_samps, 99.9)
        with z(_range_x=(0, top)):
            z.distr(bg_samps, _vertical=500.0, _percentiles=(0, 25, 50, 75, 99.9), f_x_axis_label="Intensity", f_y_axis_label="Cycle", f_title="background pixels only")
            z.distr(fg_samps, _vertical=500.0, _percentiles=(0, 25, 50, 75, 99.9), f_x_axis_label="Intensity", f_y_axis_label="Cycle", f_title="foreground pixels only")
    
        bg_medians = np.nanmedian(bg_samps, axis=1)
        fg_medians = np.nanmedian(fg_samps, axis=1)
        z.scat(
            x=fg_medians,
            y=bg_medians,
            f_x_axis_label="FG Med. Intensity",
            f_y_axis_label="BG Med. Intensity",
            _label=cy_labels,
        )
    

In [ ]:
# Experiment in power analysis
# from plaster.tools.image.coord import HW, ROI, WH, XY, YX, clip2d
# def lf_power(im, dim_half=100):
#     cen = YX(im.shape) / 2

#     a = np.copy(im)
#     a -= np.mean(a)
#     af = np.fft.fft2(a)
    
#     peak_im = imops.gauss2_rho_form(1.0, 1.2, 1.2, 5.5, 5.5, 0.0, 0.0, 11)
#     b = imops.convolve(a, peak_im)
    
#     # z.hist(b, _bins=(-400, 400, 200))
#     lft = data.one_sided_nanstd(b.flatten(), mean=0.0, negative_side=True)
#     b = np.where(b > 2*lft, np.nan, a)

# # TODO:
# # Where b in nan fill with noise

#     with z(_cols=3, _cspan=(0, 400)):
#         z.im(a)
#         z.im(b)
    
#     power = np.abs(np.fft.fftshift(b))
#     # power[power == 0] = 1

#     dim = HW(dim_half * 2 + 1, dim_half * 2 + 1)
#     roi = ROI(cen, dim, center=True)
#     spec = power[roi]
#     z.im(spec)
# #     eigen = eigen_moments(im)
# #     score = power.sum() / np.sqrt(eigen.sum())
# #     return score

# im = run.sigproc_v2.aln_unfilt_ims[0,0,0]
# lf_power(im)
